In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
import torch_explain as te
from torch_explain.logic.nn import entropy
from torch_explain.logic.metrics import test_explanation, complexity

import numpy as np
import pandas as pd
from pytorch_lightning.utilities.seed import seed_everything
from scipy.spatial.distance import cdist
from sympy import to_dnf, lambdify
from sklearn.metrics.cluster import homogeneity_score, completeness_score

import clustering_utils
import data_utils
import lens_utils
import model_utils
import persistence_utils
import visualisation_utils
import time

In [3]:
# constants
DATASET_NAME = "Tree_Cycle"
MODEL_NAME = f"GCN for {DATASET_NAME}"
NUM_CLASSES = 2
K = 10

TRAIN_TEST_SPLIT = 0.8

NUM_HIDDEN_UNITS = 10
EPOCHS = 7000
LR = 0.001

RANDOM_STATE = 0

NUM_NODES_VIEW = 5
NUM_EXPANSIONS = 4

LAYER_NUM = 2
LAYER_KEY = "conv2"

visualisation_utils.set_rc_params()

In [4]:
# model definition
class GCN(nn.Module):
    def __init__(self, num_in_features, num_hidden_features, num_classes):
        super(GCN, self).__init__()
        
        self.conv0 = DenseGCNConv(num_in_features, num_hidden_features)
        self.conv1 = DenseGCNConv(num_hidden_features, num_hidden_features)
        self.conv2 = DenseGCNConv(num_hidden_features, num_hidden_features)
                
        # linear layers
        self.lens = torch.nn.Sequential(te.nn.EntropyLinear(num_hidden_features, 1, n_classes=num_classes))

    def forward(self, x, edge_index):
        x = self.conv0(x, edge_index)
        x = F.leaky_relu(x)

        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x)

        x = self.conv2(x, edge_index)
        x = F.leaky_relu(x)
        
        x = x.squeeze()
                
        self.gnn_embedding = x
        
        x = F.softmax(x, dim=-1)
        x = torch.div(x, torch.max(x, dim=-1)[0].unsqueeze(1))
        concepts = x
        
        x = self.lens(x)
                
        return concepts, x.squeeze(-1)

In [5]:
def run_experiment(seed, path):
    config = {'seed': seed,
                       'dataset_name': DATASET_NAME,
                       'model_name': MODEL_NAME,
                       'num_classes': NUM_CLASSES,
                       'k': K,
                       'train_test_split': TRAIN_TEST_SPLIT,
                       'num_hidden_units': NUM_HIDDEN_UNITS,
                       'epochs': EPOCHS,
                       'lr': LR,
                       'num_nodes_view': NUM_NODES_VIEW,
                       'num_expansions': NUM_EXPANSIONS,
                       'layer_num': LAYER_NUM,
                       'layer_key': LAYER_KEY
                      }
        
    # load data
    G, labels = data_utils.load_syn_data(DATASET_NAME)
    data = data_utils.prepare_syn_data(G, labels, TRAIN_TEST_SPLIT, if_adj=True)

    # model training
    model = GCN(data["x"].shape[1], NUM_HIDDEN_UNITS, NUM_CLASSES)
    
    # register hooks to track activation
    model = model_utils.register_hooks(model)
    
    # train 
    start_time = time.time()
    train_acc, test_acc, train_loss, test_loss = model_utils.train(model, data, EPOCHS, LR)
    print("End time ", (time.time() - start_time))
        
#     visualisation_utils.plot_model_accuracy(train_acc, test_acc, MODEL_NAME, path)
#     visualisation_utils.plot_model_loss(train_loss, test_loss, MODEL_NAME, path)
    
#     x = data["x"]
#     edges = data['edges']
#     edges_t = data['edge_list'].numpy()
#     y = data["y"]
#     train_mask = data["train_mask"]
#     test_mask = data["test_mask"]
    
#     # get model activations for complete dataset
#     concepts, _ = model(x, edges)
#     activation = torch.squeeze(model_utils.activation_list[LAYER_KEY]).detach().numpy()
    
#     # find centroids
#     centroids, centroid_labels, used_centroid_labels = clustering_utils.find_centroids(activation, concepts, y)
#     print(f"Number of cenroids: {len(centroids)}")
    
#     # plot concept heatmaps
#     visualisation_utils.plot_concept_heatmap(centroids, activation, y, used_centroid_labels, MODEL_NAME, LAYER_NUM, path)
    
#     # concept alignment
#     homogeneity = homogeneity_score(y, used_centroid_labels)
#     # clustering efficency
#     completeness = completeness_score(y, used_centroid_labels)
    
#     print(f"Concept homogeneity score: {homogeneity}")
#     print(f"Concept completeness score: {completeness}")
    
#     # generate explanations
#     explanations = lens_utils.explain_classes(model, concepts, y, train_mask, test_mask)
    
#     # plot clustering
#     visualisation_utils.plot_clustering(seed, activation, y, centroids, centroid_labels, used_centroid_labels, MODEL_NAME, LAYER_NUM, path)
    
#     # calculate cluster sizing
#     cluster_counts = visualisation_utils.print_cluster_counts(used_centroid_labels)

#     # plot samples
#     sample_graphs, sample_feat = visualisation_utils.plot_samples(None, activation, y, LAYER_NUM, len(centroids), "Differential Clustering", "Raw", NUM_NODES_VIEW, edges_t, NUM_EXPANSIONS, path, concepts=centroids)
    
#     # dump data
#     persistence_utils.persist_experiment(config, path, 'config.z')
#     persistence_utils.persist_experiment(data, path,'data.z')
    
#     persistence_utils.persist_model(model, path, 'model.z')
    
#     persistence_utils.persist_experiment(concepts, path, 'concepts.z')
#     persistence_utils.persist_experiment(model_utils.activation_list, path, 'activation_list.z')
#     persistence_utils.persist_experiment(centroids, path, 'centroids.z')
#     persistence_utils.persist_experiment(centroid_labels, path, 'centroid_labels.z')
#     persistence_utils.persist_experiment(used_centroid_labels, path, 'used_centroid_labels.z')
    
#     concept_metrics = [('homogeneity', homogeneity), ('completeness', completeness), ('cluster_count', cluster_counts)]
#     persistence_utils.persist_experiment(concept_metrics, path, 'concept_metrics.z')
#     persistence_utils.persist_experiment(explanations, path, 'explanations.z')
#     persistence_utils.persist_experiment(sample_graphs, path, 'sample_graphs.z')
#     persistence_utils.persist_experiment(sample_feat, path, 'sample_feat.z')
    
#     # clean up
#     plt.close()

In [6]:
# run multiple times for confidence interval - seeds generated using Google's random number generator
random_seeds = [42, 19, 76, 58, 92]

for seed in random_seeds:
    print("\nSTART EXPERIMENT-----------------------------------------\n")
    seed_everything(seed)
    
    path = os.path.join("..", "output", DATASET_NAME, f"seed_{seed}_v2")
    data_utils.create_path(path)

    run_experiment(seed, path)
    
    print("\nEND EXPERIMENT-------------------------------------------\n")

Global seed set to 42



START EXPERIMENT-----------------------------------------

Task: Node Classification
Number of features:  871
Number of labels:  871
Number of classes:  871
Number of edges:  871


Global seed set to 19


End time  330.9272768497467 Train Acc: 0.95080, Test Acc: 0.96667

END EXPERIMENT-------------------------------------------


START EXPERIMENT-----------------------------------------

Task: Node Classification
Number of features:  871
Number of labels:  871
Number of classes:  871
Number of edges:  871


Global seed set to 76


End time  317.84828901290894Train Acc: 0.93566, Test Acc: 0.96154

END EXPERIMENT-------------------------------------------


START EXPERIMENT-----------------------------------------

Task: Node Classification
Number of features:  871
Number of labels:  871
Number of classes:  871
Number of edges:  871


Global seed set to 58


End time  320.0618739128113 Train Acc: 0.96017, Test Acc: 0.91667

END EXPERIMENT-------------------------------------------


START EXPERIMENT-----------------------------------------

Task: Node Classification
Number of features:  871
Number of labels:  871
Number of classes:  871
Number of edges:  871


Global seed set to 92


End time  320.18176794052124Train Acc: 0.94781, Test Acc: 0.95679

END EXPERIMENT-------------------------------------------


START EXPERIMENT-----------------------------------------

Task: Node Classification
Number of features:  871
Number of labels:  871
Number of classes:  871
Number of edges:  871
End time  330.0943212509155 Train Acc: 0.95461, Test Acc: 0.94681

END EXPERIMENT-------------------------------------------

